In [1]:
from cProfile import run
from dask.distributed import Client, LocalCluster
import time
import json

from dask.distributed import Client
import numpy as np
import dask.dataframe as dd

/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
user_reviews_csv = dd.read_csv('~/data/user_reviews_Release.csv', converters={'helpful': lambda x: eval(x)})

user_reviews_csv.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,Gerald DeWitt,"[0, 0]",Beware! This is NOT the original single versi...,1.0,Poor Quality Alternate Take,1.400630e+09,"05 21, 2014"
1,A3TYW0XA8HSGWB,B00EKR5S0Q,Linda E. Larson,"[0, 0]",This is my new most favorite k-cup coffee. I c...,5.0,Vanilla Starbucks K-cups,1.398557e+09,"04 27, 2014"
2,A2CME0TQU2IVVB,B001AUPJVO,L5Momma,"[1, 1]",This headset is great! It worked in our 2007 ...,5.0,Awesome!,1.355875e+09,"12 19, 2012"
3,A2E5IDLX7R388S,B000055Y57,Jeff Andersen,"[0, 0]",Scofield is one of my favorite musicians and i...,5.0,Straight ahead Jazz with the Scofield twist,1.402358e+09,"06 10, 2014"
4,A3CIEMYUGV6ZMR,0545265355,Adroit,"[0, 0]","Wonderful book! I cried, well teared up at a f...",5.0,Greatest Book Ever!!!,1.334102e+09,"04 11, 2012"


In [3]:
dtypes = {
    'asin': str,
    'salesRank': object,
    'imUrl': str,
    'categories': object,
    'title': str,
    'description': str,
    'price': float,
    'related': object,
    'brand': str
    }

products_csv = dd.read_csv('~/data/products_Release.csv', dtype=dtypes)

products_csv.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,0001048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0000143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
2,0000037214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams
3,0000032069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi
4,0000031909,{'Toys & Games': 201847},http://ecx.images-amazon.com/images/I/41xBoP0F...,"[['Sports & Outdoors', 'Other Sports', 'Dance']]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown


In [8]:
# read data 
# products_csv = dd.read_csv(products_csv)
# user_reviews_csv = dd.read_csv(user_reviews_csv)
combined = products_csv[['asin', 'price']].merge(user_reviews_csv[['asin', 'overall']], on=['asin'])

In [4]:
q1_reviews = (user_reviews_csv.isna().mean() * 100).compute()
q1_products = (products_csv.isna().mean() * 100).compute()

In [5]:
q1_reviews

reviewerID        0.000000
asin              0.000000
reviewerName      3.292575
helpful           0.000000
reviewText        0.021570
overall           0.000000
summary           0.001085
unixReviewTime    0.000010
reviewTime        0.000000
dtype: float64

In [6]:
q1_products

asin            0.000000
salesRank      26.015663
imUrl           1.795239
categories      0.798038
title          15.192672
description    41.717847
price          35.703404
related        29.039236
brand          81.684125
dtype: float64

In [5]:
# q1
reviews_shape = user_reviews_csv.shape
products_shape = products_csv.shape
q1_reviews = 100*(reviews_shape[0] - user_reviews_csv.count()) / reviews_shape[0]
q1_products = 100*(products_shape[0] - products_csv.count()) / products_shape[0]

In [ ]:
q1_reviews.compute()

In [ ]:
# q2
q2 = combined.corr(method = 'pearson').compute()

In [7]:
# q3
q3 = combined.price.describe().compute()[['mean', 'std', '50%', 'min', 'max']]

In [5]:
# q4
q4 = products_csv.categories.dropna().apply(lambda x: eval(x)[0][0], meta=('categories', 'object')).value_counts().compute()

In [6]:
q4

Books                                       2369910
Clothing, Shoes & Jewelry                   1435868
Sports & Outdoors                            529989
Electronics                                  495476
CDs & Vinyl                                  491713
                                             ...   
Publishers                                        2
Celebrate your Birthday with Nickelodeon          2
Gospel                                            2
Collectible Coins                                 1
#508510                                           1
Name: categories, Length: 83, dtype: int64

In [ ]:
# q5
user_reviews_csv.asin.isin(products_csv.asin.tolist())

In [ ]:
# data extract
pool = sum(products_csv.related.apply(lambda x: sum(x.values(), [])), [])
user_reviews_csv.asin.isin(pool)